In [1]:
!python --version

Python 3.10.12


In [2]:
!pip install cornac

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 13.8 MB/s eta 0:00:00


In [3]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [4]:
FILE_PREFIX="."
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  FILE_PREFIX="/content/drive/MyDrive/daicon"


Mounted at /content/drive


In [5]:
# !cp -Rf  /usr/local/lib/python3.10/dist-packages/cornac  /content/drive/MyDrive/daicon

In [6]:
import sys
# sys.path.insert(0,'/content/drive/MyDrive/daicon/msr')
if FILE_PREFIX not in sys.path:
  sys.path.insert(0,FILE_PREFIX)


# if "/content/drive/MyDrive/daicon/cornac" not in sys.path:
#   sys.path.insert(0,'/content/drive/MyDrive/daicon/cornac')

# if "/content/drive/MyDrive/daicon/cornac/cornac" not in sys.path:
#   sys.path.insert(0,'/content/drive/MyDrive/daicon/cornac/cornac')

print (f"{sys.path}")
import torch
import pandas as pd
import numpy as np
import cornac

from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit,StratifiedSplit,CrossValidation

from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch
from sklearn.model_selection import GridSearchCV


from msr.python_splitters import python_stratified_split
from msr.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from msr.cornac_utils import predict_ranking


from cornac.data import FeatureModality

print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

['/content/drive/MyDrive/daicon', '/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']
System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Cornac version: 1.17


In [7]:
new_apply_train = pd.read_csv(f'{FILE_PREFIX}/data/apply_train.csv')
new_apply_train['rating']=1.0
new_apply_train.columns =["userID", "itemID", "rating"]

In [8]:
train, test = python_stratified_split(new_apply_train, 0.90)


In [9]:
SEED=42
BEST_COL=7

In [10]:
train_tuple = list(train.itertuples(index=False,name=None))
test_tuple = list(test.itertuples(index=False,name=None))
train_all_tuple = list(new_apply_train.itertuples(index=False,name=None))


train_ds = cornac.data.Dataset.from_uir(train_tuple, seed=SEED)
test_ds = cornac.data.Dataset.from_uir(test_tuple, seed=SEED)

train_all_ds = cornac.data.Dataset.from_uir(train_all_tuple, seed=SEED)


## RESUME

In [11]:
resume_go = pd.read_csv(f'{FILE_PREFIX}/data/resume.csv')
# resume_go = pd.merge(resume_go, pd.read_csv(f'{FILE_PREFIX}/data/resume_certificate.csv'),on='resume_seq')
# resume_go = pd.merge(resume_go, pd.read_csv(f'{FILE_PREFIX}/data/resume_education.csv'),on='resume_seq')
# resume_go = pd.merge(resume_go, pd.read_csv(f'{FILE_PREFIX}/data/resume_language.csv'),on='resume_seq')


In [12]:
resume_go.head(3)


,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,NaN,67,NaN
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,NaN,84,섬유;봉제;가방;의류
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,NaN,121,학교;학원;직원훈련(교육서비스)


In [13]:
resume_go = resume_go.dropna(axis=1, how='any')
resume_go.head(3)


,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,job_code_seq1,career_month
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,재료·화학·섬유·의복,67
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,재료·화학·섬유·의복,84
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,재료·화학·섬유·의복,121


In [14]:
resume_go.columns

Index(['resume_seq', 'reg_date', 'updated_date', 'degree', 'graduate_date',
       'hope_salary', 'last_salary', 'job_code_seq1', 'career_month'],
      dtype='object')

In [15]:
resume_go2 = resume_go[['resume_seq','degree']]
resume_go2

,resume_seq,degree
0,U00606,4
1,U00509,2
2,U02012,5
3,U04599,4
4,U07573,4
...,...,...
8477,U04890,5
8478,U05910,4
8479,U01468,4
8480,U05315,4


In [16]:
resume_go2_enc = pd.get_dummies(resume_go2,columns=['degree'])
resume_go2_enc

,resume_seq,degree_2,degree_3,degree_4,degree_5,degree_6
0,U00606,0,0,1,0,0
1,U00509,1,0,0,0,0
2,U02012,0,0,0,1,0
3,U04599,0,0,1,0,0
4,U07573,0,0,1,0,0
...,...,...,...,...,...,...
8477,U04890,0,0,0,1,0
8478,U05910,0,0,1,0,0
8479,U01468,0,0,1,0,0
8480,U05315,0,0,1,0,0


In [17]:
resume_go2['resume_seq'].nunique()

8482

In [18]:
# uf_count = new_apply_train['userID'].nunique()-1
# t=6
# a=np.zeros([1, t])
# for _ in range(uf_count):
#   a=np.vstack([a, np.zeros([1, t], dtype=np.float32)])

# resume_go2_enc
# # resume_go2_enc
# for id in new_apply_train['userID']:
#   new_df = resume_go2_enc[resume_go2_enc['resume_seq']==id]

# df_merged = pd.merge(new_apply_train, resume_go2_enc, left_on='userID', right_on='resume_seq', how='left').fillna(0)
# df_merged
a=np.zeros_like(resume_go2_enc.iloc[:,1:].values.astype(np.float32))
a= np.zeros(((resume_go2_enc.iloc[:,1:].shape[0]),BEST_COL))

a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
uf_modality = FeatureModality(a,resume_go2_enc['resume_seq'].map(train_ds.uid_map))


## COMPANY

In [20]:
# 6695
company_go = pd.read_csv(f'{FILE_PREFIX}/data/recruitment.csv')
# company_go = pd.merge(company_go, pd.read_csv(f'{FILE_PREFIX}/data/company.csv'),on='recruitment_seq')
company_go

,recruitment_seq,address_seq1,address_seq2,address_seq3,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword
0,R02264,3.0,NaN,NaN,0,0,2507;2707;2810,4,8,1,NaN
1,R06317,3.0,NaN,NaN,0,0,2204;2205;2707,3,2,1,NaN
2,R04017,3.0,NaN,NaN,0,0,2101;2108;2201;2707,3,2,1,NaN
3,R02865,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,1,NaN
4,R04890,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6690,R03678,3.0,NaN,NaN,0,0,2101;2108;2201;2204;2205;2707,3,2,1,NaN
6691,R04593,3.0,NaN,NaN,0,0,2201;2204;2205;2707,4,2,1,NaN
6692,R03252,3.0,NaN,NaN,0,0,2109,3,2,1,NaN
6693,R05130,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,2,NaN


In [21]:
company_go.columns

Index(['recruitment_seq', 'address_seq1', 'address_seq2', 'address_seq3',
       'career_end', 'career_start', 'check_box_keyword', 'education',
       'major_task', 'qualifications', 'text_keyword'],
      dtype='object')

In [22]:
company_go2=company_go[['recruitment_seq','education','major_task','qualifications']]
company_go2

,recruitment_seq,education,major_task,qualifications
0,R02264,4,8,1
1,R06317,3,2,1
2,R04017,3,2,1
3,R02865,2,2,1
4,R04890,2,2,2
...,...,...,...,...
6690,R03678,3,2,1
6691,R04593,4,2,1
6692,R03252,3,2,1
6693,R05130,2,2,2


In [23]:
company_go2_enc = pd.get_dummies(company_go2,columns=['education','major_task','qualifications'])
company_go2_enc

,recruitment_seq,education_2,education_3,education_4,education_6,major_task_1,major_task_2,major_task_6,major_task_7,major_task_8,major_task_9,major_task_10,qualifications_0,qualifications_1,qualifications_2
0,R02264,0,0,1,0,0,0,0,0,1,0,0,0,1,0
1,R06317,0,1,0,0,0,1,0,0,0,0,0,0,1,0
2,R04017,0,1,0,0,0,1,0,0,0,0,0,0,1,0
3,R02865,1,0,0,0,0,1,0,0,0,0,0,0,1,0
4,R04890,1,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,R03678,0,1,0,0,0,1,0,0,0,0,0,0,1,0
6691,R04593,0,0,1,0,0,1,0,0,0,0,0,0,1,0
6692,R03252,0,1,0,0,0,1,0,0,0,0,0,0,1,0
6693,R05130,1,0,0,0,0,1,0,0,0,0,0,0,0,1


In [24]:
# b=np.zeros_like(company_go2_enc.iloc[:,1:].values.astype(np.float32))
b= np.zeros(((company_go2_enc.iloc[:,1:].shape[0]),BEST_COL))
b


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
company_go['recruitment_seq'].nunique()

6695

In [26]:
company_go['recruitment_seq'].map(train_ds.iid_map)

0       3367.0
1       1520.0
2       3967.0
3       3757.0
4       6639.0
         ...  
6690     564.0
6691    1324.0
6692    4846.0
6693    5434.0
6694     940.0
Name: recruitment_seq, Length: 6695, dtype: float64

In [27]:
if_modality = FeatureModality(b,company_go['recruitment_seq'].map(train_ds.iid_map))


## END COMPANY

In [28]:
SEED=42

In [29]:
len(train)

52801

In [30]:
train.head(2)

,userID,itemID,rating
11989,U00001,R05210,1.0
51174,U00001,R04536,1.0


In [31]:
from itertools import islice

for i, row in enumerate(islice(train.itertuples(index=False,name=None), 3)):
    print(i, row)

for i, row in enumerate(islice(train_tuple,3)):
    print(i, row)


0 ('U00001', 'R05210', 1.0)
1 ('U00001', 'R04536', 1.0)
2 ('U00001', 'R05288', 1.0)
0 ('U00001', 'R05210', 1.0)
1 ('U00001', 'R04536', 1.0)
2 ('U00001', 'R05288', 1.0)


In [32]:
#1100   1400 1500 1700

In [33]:
LATENT_DIM = 256
ENCODER_DIMS = [512]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
# NUM_EPOCHS = 2000
NUM_EPOCHS = 1100
BATCH_SIZE = 128
LEARNING_RATE = 0.001
SEED=42
BETA_KL=1.0
TOP_K=5

In [34]:
# uf_count = new_apply_train['itemID'].nunique()-1
# b=np.zeros([1, t])
# for _ in range(uf_count):
#   b=np.vstack([b, np.zeros([1, t], dtype=np.float32)])

In [35]:
print("_______________________")
print(type(list(train.itertuples(index=False, name=None))[0]))
print("_______________________")
print(type(uf_modality.features[0]))
print("_______________________")
print(type(train_tuple))
print("_______________________")


_______________________
<class 'tuple'>
_______________________
<class 'numpy.ndarray'>
_______________________
<class 'list'>
_______________________


In [36]:
eval_method = cornac.eval_methods.base_method.BaseMethod.from_splits(
    train_data=(train_tuple),
    test_data=(test_tuple),
    seed=SEED,
    fmt='UIR',
    user_feature = uf_modality,
    item_feature = if_modality,
    rating_threshold =0.5,
    verbose=True,
)

rating_threshold = 0.5
exclude_unknowns = False
---
Training data:
Number of users = 8482
Number of items = 6694
Number of ratings = 52801
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 3922
Number of items = 3426
Number of ratings = 5145
Number of unknown users = 0
Number of unknown items = 1
---
Total users = 8482
Total items = 6695


In [37]:
# Instantiate models
# import importlib
# importlib.reload(cornac.models)
from  cornac.models import BiVAECF,EASE
# epoch=[1100,1400,1500,1700]
epoch=[2000]
# epoch=[1,2]

models={}
for epo in epoch:
  model = BiVAECF(
      k=LATENT_DIM,
      encoder_structure=ENCODER_DIMS,
      act_fn=ACT_FUNC,
      likelihood=LIKELIHOOD,
      n_epochs=epo,
      batch_size=BATCH_SIZE,
      learning_rate=LEARNING_RATE,
      seed=SEED,
      use_gpu=torch.cuda.is_available(),
      verbose=True,
      beta_kl=BETA_KL,
      cap_priors = {"user":False,"item":False},
      name=f"BiVAECF-{epo}"
  )
  models[model.name]=(model)

params = {
    'lamb': 150,
    'posB': True,
    'name': 'EASE'
}

model = EASE(**params, seed=SEED, verbose=True)
models[model.name] = (model)


for model in models:
  print(model)



BiVAECF-2000
EASE


In [38]:
uf_modality.feature_dim

7

In [39]:
for i in models.values():
  print(i)

In [40]:


from cornac.metrics import MAE, RMSE, Precision, Recall, NDCG, AUC, MAP
MULTIPLE=1
topk_metrics = [cornac.metrics.NDCG(k=TOP_K*MULTIPLE), cornac.metrics.Recall(k=TOP_K*MULTIPLE)]

exp = cornac.Experiment(
    eval_method=eval_method,
                        models=[m for m in models.values()],
                        metrics=topk_metrics,
                        user_based=True
                        )
exp.run()
# BiVAECF | 0.0000 |      0.0007 |   0.0025 |  181.2209 |   3.5680



[BiVAECF-2000] Training started!


  0%|          | 0/2000 [00:00<?, ?it/s]


[BiVAECF-2000] Evaluation started!


Ranking:   0%|          | 0/3922 [00:00<?, ?it/s]


[EASE] Training started!

[EASE] Evaluation started!


Ranking:   0%|          | 0/3922 [00:00<?, ?it/s]


TEST:
...
             | NDCG@5 | Recall@5 | Train (s) | Test (s)
------------ + ------ + -------- + --------- + --------
BiVAECF-2000 | 0.0839 |   0.1255 | 1849.1180 |   5.0694
EASE         | 0.0767 |   0.1149 |    5.4942 |   4.9324



In [41]:
train_all_ds.add_modalities(user_feature=uf_modality, item_feature=if_modality)


In [42]:
list(exp.result[0].metric_avg_results.keys())[1]


'Recall@5'

In [43]:
exp.result[1].metric_avg_results[list(exp.result[0].metric_avg_results.keys())[1]]

0.11485059857701363

In [44]:
for idx,name in enumerate(models.keys()):

  mymodel = models[name]
  print(mymodel.name)
  mymodel.fit(train_all_ds)

  score = exp.result[idx].metric_avg_results[list(exp.result[0].metric_avg_results.keys())[1]]
  all_predictions = predict_ranking(mymodel, new_apply_train, usercol='userID', itemcol='itemID', remove_seen=True)
  all_predictions=all_predictions.sort_values(by=['userID','prediction'],ascending=[True,False])
  all_predictions['rank'] = all_predictions.groupby('userID').cumcount()+1
  print(score)
  print(all_predictions.iloc[0:10])

  all_predictions.to_csv(f'{FILE_PREFIX}/temp_result/{mymodel.name}_{score}.csv', index=False,header=True)


BiVAECF-2000


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/8482 [00:00<?, ?it/s]

0.125459556591632
         userID  itemID    prediction  rank
2277370  U00001  R03811  1.159507e-03     1
2280513  U00001  R00165  1.059652e-03     2
2276346  U00001  R01528  9.486059e-04     3
2280206  U00001  R06276  3.540045e-05     4
2275965  U00001  R02888  1.307977e-05     5
2277914  U00001  R03777  9.706588e-06     6
2277523  U00001  R03021  8.177786e-06     7
2276917  U00001  R02064  4.277022e-06     8
2279690  U00001  R01473  9.350172e-07     9
2279615  U00001  R03037  7.951176e-07    10
EASE


  0%|          | 0/8482 [00:00<?, ?it/s]

0.11485059857701363
         userID  itemID  prediction  rank
2276346  U00001  R01528    0.037793     1
2277370  U00001  R03811    0.031134     2
2279615  U00001  R03037    0.029968     3
2280206  U00001  R06276    0.029069     4
2275965  U00001  R02888    0.027442     5
2277298  U00001  R04769    0.026737     6
2280513  U00001  R00165    0.024347     7
2277181  U00001  R00516    0.024117     8
2276976  U00001  R04708    0.023165     9
2277914  U00001  R03777    0.022690    10
